In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import sys
import os
import umap

from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.append("../profiling/")
import profiling

/home/jccaicedo/.local/lib/python3.6/site-packages/numba/errors.py:131: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
PROJECT_ROOT = "/storage/data/cellpainting/BBBC022/dp-project/"
EXP = "results"
NUM_FEATURES = 7680 #6400
OUTPUT_FILE = "data/well_level_data.csv"

In [3]:
# Load metadata
meta = pd.read_csv(os.path.join(PROJECT_ROOT, "inputs/metadata/index.csv"))

In [8]:
meta

,Unnamed: 0,Metadata_Well,Metadata_Site,Metadata_Plate,Assay_Plate_Barcode,Plate_Map_Name,DNA,ER,RNA,AGP,Mito,broad_sample,broad_sample_Replicate,Treatment,Compound,Concentration
0,0,A01,1,20585,20585,H-BIOA-002-1,20585/IXMtest_A01_s1_w164FBEEF7-F77C-4892-86F5...,20585/IXMtest_A01_s1_w22CA7BDF3-1425-4AE6-8E57...,20585/IXMtest_A01_s1_w3FD54227A-6BE4-460B-8E2C...,20585/IXMtest_A01_s1_w47457436C-5BE7-4DA2-B966...,20585/IXMtest_A01_s1_w58DA6AF62-1799-4B85-BE17...,0,1,BRD-K98763141-001-06-8@8.858401197993738,BRD-K98763141-001-06-8,8.858401
1,1,A01,2,20585,20585,H-BIOA-002-1,20585/IXMtest_A01_s2_w1AA6B1894-F561-42EE-9D1D...,20585/IXMtest_A01_s2_w246FFAEE1-BEB6-4C81-913B...,20585/IXMtest_A01_s2_w3A597237B-C3D7-43AE-8399...,20585/IXMtest_A01_s2_w46657239A-5AFE-4C29-BB9B...,20585/IXMtest_A01_s2_w50F1562CD-EBCF-408E-9D8D...,0,1,BRD-K98763141-001-06-8@8.858401197993738,BRD-K98763141-001-06-8,8.858401
2,2,A01,3,20585,20585,H-BIOA-002-1,20585/IXMtest_A01_s3_w108F3A6FF-A865-40A1-8488...,20585/IXMtest_A01_s3_w22AD5BD08-58A3-466D-A364...,20585/IXMtest_A01_s3_w30B946780-C923-4A40-8EC5...,20585/IXMtest_A01_s3_w47B5F2FF6-A5A2-4101-AC2A...,20585/IXMtest_A01_s3_w5489716D4-2D06-45BE-8D68...,0,1,BRD-K98763141-001-06-8@8.858401197993738,BRD-K98763141-001-06-8,8.858401
3,3,A01,4,20585,20585,H-BIOA-002-1,20585/IXMtest_A01_s4_w189967984-D8CB-4449-9CE5...,20585/IXMtest_A01_s4_w231D1EC99-22A8-47A8-9691...,20585/IXMtest_A01_s4_w3F582BADD-2249-48D6-BC19...,20585/IXMtest_A01_s4_w47E34AB5F-23B4-42E1-8B5B...,20585/IXMtest_A01_s4_w5D0566314-B9B2-4299-98AC...,0,1,BRD-K98763141-001-06-8@8.858401197993738,BRD-K98763141-001-06-8,8.858401
4,4,A01,5,20585,20585,H-BIOA-002-1,20585/IXMtest_A01_s5_w1DCC72C3D-6ABF-4D49-897E...,20585/IXMtest_A01_s5_w2B3F07F02-876D-455B-94E4...,20585/IXMtest_A01_s5_w3EB7ABD45-EE05-41A0-979E...,20585/IXMtest_A01_s5_w4AFA01300-7373-42C4-872F...,20585/IXMtest_A01_s5_w513201EB0-3500-4A0C-8FAC...,0,1,BRD-K98763141-001-06-8@8.858401197993738,BRD-K98763141-001-06-8,8.858401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69115,69115,P24,5,20646,20646,H-BIOA-006-1,20646/IXMtest_P24_s5_w165703D98-F910-4E5C-93C6...,20646/IXMtest_P24_s5_w23EE5080A-0B6E-4AE1-A413...,20646/IXMtest_P24_s5_w36BAFF86A-1994-4895-818F...,20646/IXMtest_P24_s5_w4FC9030FD-81D1-43DF-AA8B...,20646/IXMtest_P24_s5_w5811D5170-11AC-4A60-8AFF...,1600,4,BRD-K08832567-001-02-4@5.0,BRD-K08832567-001-02-4,5.000000
69116,69116,P24,6,20646,20646,H-BIOA-006-1,20646/IXMtest_P24_s6_w184D3491C-2361-479E-A11B...,20646/IXMtest_P24_s6_w2F9235A94-86DA-4703-B46B...,20646/IXMtest_P24_s6_w3CC6BB025-8969-4F85-B30E...,20646/IXMtest_P24_s6_w4BD3DA210-B3C7-4371-B06D...,20646/IXMtest_P24_s6_w5300E0A76-48E7-489A-AEC7...,1600,4,BRD-K08832567-001-02-4@5.0,BRD-K08832567-001-02-4,5.000000
69117,69117,P24,7,20646,20646,H-BIOA-006-1,20646/IXMtest_P24_s7_w1B04E085D-04AB-42F7-8D08...,20646/IXMtest_P24_s7_w209CDD7FD-92F0-40B4-85C6...,20646/IXMtest_P24_s7_w329876479-D0F9-4A3F-BD44...,20646/IXMtest_P24_s7_w4DBB3B5B7-0EF6-4418-97D7...,20646/IXMtest_P24_s7_w55DE4D5D3-E291-4FEB-A3BE...,1600,4,BRD-K08832567-001-02-4@5.0,BRD-K08832567-001-02-4,5.000000
69118,69118,P24,8,20646,20646,H-BIOA-006-1,20646/IXMtest_P24_s8_w166C6499D-6DE8-4BDB-90CE...,20646/IXMtest_P24_s8_w23695868E-F144-4CDF-B1A6...,20646/IXMtest_P24_s8_w3F65034AA-2A45-470A-8F98...,20646/IXMtest_P24_s8_w4D4765757-C520-4B74-9301...,20646/IXMtest_P24_s8_w5090B2268-B603-491F-B313...,1600,4,BRD-K08832567-001-02-4@5.0,BRD-K08832567-001-02-4,5.000000


In [ ]:
#meta["broad_sample"] = meta["Treatment"].str.split("@", expand=True)[0]

# 1. Load single-cell data

In [5]:
features = []
for i in tqdm(meta.index):
    filename = PROJECT_ROOT + "outputs/" + EXP + "/features.old/{}_{}_{}.npz"
    filename = filename.format(
        meta.loc[i, "Metadata_Plate"], 
        meta.loc[i, "Metadata_Well"], 
        meta.loc[i, "Metadata_Site"]
    )
    if os.path.isfile(filename):
        with open(filename, "rb") as data:
            info = np.load(data)
            features.append(info["f"])
    else:
        features.append([])

100%|██████████| 69120/69120 [1:09:58<00:00, 16.46it/s]


In [6]:
total_single_cells = 0
for i in range(len(features)):
    if len(features[i]) > 0:
        total_single_cells += features[i].shape[0]

print("Total images",len(features),features[0].shape)
print("Total single cells:", total_single_cells)

Total images 69120 (35, 7680)
Total single cells: 3410627


# 2. Site-level profiles / Median Aggregation

In [9]:
site_level_data = []
site_level_features = []
for plate in tqdm(meta["Metadata_Plate"].unique()):
    m1 = meta["Metadata_Plate"] == plate
    wells = meta[m1]["Metadata_Well"].unique()
    for well in wells:
        result = meta.query("Metadata_Plate == '{}' and Metadata_Well == '{}'".format(plate, well))
        for i in result.index:
            if len(features[i]) == 0:
                continue
            num_features = features[i].shape[1]
            mean_profile = np.median(features[i], axis=0)
            pert_name = result["Treatment"].unique()
            replicate = result["broad_sample_Replicate"].unique()
            if len(pert_name) > 1:
                print(pert_name)
            site_level_data.append(
                {
                    "Plate": plate,
                    "Well": well,
                    "Treatment": pert_name[0],
                    "Replicate": replicate[0],
                    "broad_sample": pert_name[0].split("@")[0]
                }
            )
            site_level_features.append(mean_profile)

100%|██████████| 20/20 [09:14<00:00, 27.71s/it]


In [10]:
columns1 = ["Plate", "Well", "Treatment", "Replicate", "broad_sample"]
columns2 = [i for i in range(num_features)]

sites1 = pd.DataFrame(columns=columns1, data=site_level_data)
sites2 = pd.DataFrame(columns=columns2, data=site_level_features)
sites = pd.concat([sites1, sites2], axis=1)

# 3. Well-level profiles / Mean Aggregation

In [30]:
# Collapse well data
wells = sites.groupby(["Plate", "Well", "Treatment"]).mean().reset_index()

tmp = meta.groupby(["Metadata_Plate", "Metadata_Well", "Treatment", "broad_sample"])["DNA"].count().reset_index()
wells = pd.merge(wells, tmp, how="left", left_on=["Plate", "Well", "Treatment"], right_on=["Metadata_Plate", "Metadata_Well", "Treatment"])

wells = wells[columns1 + columns2]

# 4. Whitening

In [31]:
whN = profiling.WhiteningNormalizer(wells.loc[wells["Treatment"] == "NA@NA", columns2], reg_param=1e-6)

(7680,) (7680, 7680)


In [32]:
whD = whN.normalize(wells[columns2])

In [33]:
# Save whitened profiles
wells[columns2] = whD
wells.to_csv(OUTPUT_FILE, index=False)

# 5. Treatment-level profiles / Mean Aggreagation

In [34]:
# Aggregate profiles
columns1 = ["Plate", "Well", "Treatment", "Replicate", "broad_sample"]
columns2 = [i for i in range(num_features)] 
profiles = wells.groupby("Treatment").mean().reset_index()

In [35]:
wells["broad_sample"] = wells["Treatment"].str.split("@", expand=True)[0]

In [39]:
# Recover broad_sample column (cannot be used in groupby because it contains NaN values)
tmp = wells.groupby(["Treatment", "broad_sample"])["Replicate"].count().reset_index()
profiles = pd.merge(profiles.reset_index(), tmp, on="Treatment", how="left")

In [40]:
profiles

,level_0,index,Treatment,Plate,Replicate_x,broad_sample_x,0,1,2,3,...,7674,7675,7676,7677,7678,7679,broad_sample_y,Replicate_y,broad_sample,Replicate
0,0,0,BRD-A00587958-001-02-1@3.8715447469734894,20604.25,2.5,205.0,-3.015418e-03,-7.838889e-03,6.412347e-02,1.824343e-06,...,9.018333e-52,4.497169e-06,1.503327e-52,-6.907951e-52,-1.286775e-02,-4.028980e-07,BRD-A00587958-001-02-1,4,BRD-A00587958-001-02-1,4
1,1,1,BRD-A00821662-001-02-1@7.706137007224904,20612.00,2.5,829.0,5.380710e-04,-5.826178e-03,5.885512e-02,-7.867977e-06,...,-1.400395e-51,-6.430667e-04,-2.802929e-52,1.167375e-51,-2.520100e-01,1.190791e-06,BRD-A00821662-001-02-1,4,BRD-A00821662-001-02-1,4
2,2,2,BRD-A00827783-001-05-5@3.93325177342456,20612.00,2.5,730.0,1.950312e-03,1.619100e-02,-2.895427e-02,-2.781322e-05,...,-1.393052e-51,-1.764384e-05,-2.530269e-52,1.110585e-51,-4.188282e-02,-4.062505e-07,BRD-A00827783-001-05-5,4,BRD-A00827783-001-05-5,4
3,3,3,BRD-A00993607-003-05-5@3.4988332790547663,20606.50,2.5,484.0,-3.659747e-03,-2.977179e-03,2.472721e-02,-4.338930e-05,...,-3.037497e-51,1.287180e-04,-4.827002e-52,2.276351e-51,-1.476716e-02,-3.169343e-07,BRD-A00993607-003-05-5,4,BRD-A00993607-003-05-5,4
4,4,4,BRD-A01078468-001-05-6@3.6986888296046603,20606.50,2.5,555.0,9.978154e-04,-2.051746e-02,-9.073446e-02,2.596409e-05,...,-1.434754e-43,-4.470857e-04,-2.277371e-44,1.084138e-43,1.169084e-02,-9.590783e-07,BRD-A01078468-001-05-6,4,BRD-A01078468-001-05-6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,1594,1594,BRD-K99792991-001-02-6@5.0,20615.75,2.5,1484.0,1.900448e-03,-4.411534e-02,-2.443170e-01,3.774808e-04,...,1.655113e-20,1.151822e-03,-6.562668e-21,2.811121e-20,-1.419083e-01,5.125317e-04,BRD-K99792991-001-02-6,4,BRD-K99792991-001-02-6,4
1595,1595,1595,BRD-K99946902-001-02-6@5.0,20613.00,2.5,1219.0,-1.275722e-03,-1.564505e-02,-3.307892e-03,-1.613621e-05,...,1.589236e-51,-6.547051e-05,2.587757e-52,-1.204178e-51,6.851784e-02,-1.120452e-06,BRD-K99946902-001-02-6,4,BRD-K99946902-001-02-6,4
1596,1596,1596,BRD-K99984802-098-03-2@2.034761335241121,20606.50,2.5,469.0,1.283556e-03,1.086938e-02,2.541241e-02,2.169581e-05,...,3.156585e-52,-5.860095e-05,2.213398e-53,-1.797193e-52,-1.128832e-02,-4.022051e-08,BRD-K99984802-098-03-2,4,BRD-K99984802-098-03-2,4
1597,1597,1597,BRD-K99984802-100-01-0@5.0,20615.75,2.5,1540.0,4.026976e-03,-4.035049e-03,7.266059e-02,-8.381211e-06,...,-3.267525e-51,-1.487503e-04,-5.277959e-52,2.468556e-51,3.961139e-02,-5.172370e-07,BRD-K99984802-100-01-0,4,BRD-K99984802-100-01-0,4


In [41]:
profiles = profiles[["Treatment", "broad_sample"] + columns2]

In [42]:
# Remove samples without MOA (according to [1])
Y = pd.read_csv("data/BBBC022_MOA_MATCHES.csv")
profiles = pd.merge(profiles, Y, left_on="broad_sample", right_on="Var1")
profiles = profiles[["Treatment", "broad_sample", "Metadata_moa.x"] + columns2].sort_values(by="broad_sample")

# 6. Correlation matrix

In [43]:
# Compute Pearson correlation
CRM = np.corrcoef(profiles[columns2])

# Transform to tidy format
df = pd.DataFrame(data=CRM, index=list(profiles.broad_sample), columns=list(profiles.broad_sample))
df = df.reset_index().melt(id_vars=["index"])

In [44]:
# Annotate rows
df2 = pd.merge(
    df, 
    profiles[["broad_sample", "Metadata_moa.x"]], 
    how="left", 
    left_on="index", # <=== Rows
    right_on="broad_sample"
).drop("broad_sample",axis=1)

# Annotate columns
df2 = pd.merge(
    df2, profiles[["broad_sample", "Metadata_moa.x"]],
    how="left", 
    left_on="variable", # <=== Columns
    right_on="broad_sample"
).drop("broad_sample",axis=1)

In [45]:
# Rename columns and save
df2.columns = ["Var1", "Var2", "value", "Metadata_moa.x", "Metadata_moa.y"]
df2.to_csv("data/correlation_matrix.csv")